#  Usher Syndrome associated genes and the evolution of sensory structures


<img src ="files/Presentation_fig1.png">

## Find the project at github:

https://github.com/hspeck/project.git

# Step 1: BLAST sequences of Interest against NCBI Databases


In [2]:
full_taxa_file_name=open("full_list_taxa_NCBI.txt", "r")
full_taxa_list = full_taxa_file_name.read()
#read in the list of organisms to search
formated_full_taxa_list = full_taxa_list.replace("a", "").replace('"', '').replace("\n", "[ORGN]\n").replace(":", "").split("\n")
#format the organism name

### qBLAST function

In [1]:
def search_taxa_all_gene_delay(list_of_taxa):
    #takes a list of taxa and will run blast for sequences contained in a file
    #will loop through the list and run blast for each one
    #will save each result to a separate xml file
    from Bio.Blast import NCBIWWW
        #imports the NCBIWWW module from Biopython
    import time
        #gives us the ability to delay our imputs to not spam the NCBI servers and get kicked off
    
    with open("USH_Search_seq.fasta", "r") as fasta_file:
        sequences = fasta_file.read()
        fasta_file.close()
        #reads in the file we will be searching
        
    for i in list_of_taxa:
        result_handle = NCBIWWW.qblast("blastp", 
                                       "refseq_protein", 
                                       sequences, 
                                       alignments=100, 
                                       descriptions=100, 
                                       expect=0.00001, 
                                       entrez_query=str(i))
        file_name=str("USH_Search_"+str(i)+".xml") #this creates a name for the file
        save_file=open(file_name, "w")  #we are opening a file that does not yet exist to write to it
        save_file.write(result_handle.read())  #writing the result of our blast search to local file
        save_file.close() #closing it to allow the file to actually write it
        result_handle.close() #close the results handle
        print("created "+ file_name)
        time.sleep(60)  #this gives 1 minute between writing the output and sending another request to the ncbi server
            #hopefully this will prevent spamming the server 
        

#### Delete the hashtag below to search!
(be prepared to wait 5 hours)

In [3]:
#    search_taxa_all_gene_delay(formated_full_taxa_list)

# Search Output XML files and summarize search

In [4]:
def parse_and_getID(blast_output_xml):
    #  goes through the output of a BLAST xml file 
    # finds the ID for the top 5 of the result
    from Bio.Blast import NCBIXML
    from Bio.SeqRecord import SeqRecord
    #import the required libraries
    
    for taxa_id in blast_output_xml:
        file_name = str("USH_Search_"+taxa_id+"[ORGN].xml")
        #this sets the handle, without having to format every single name of the input files
        
        result_handle = open(str(file_name), "r") #sets the result handle
        blast_records = NCBIXML.parse(result_handle)
        #need to use parse if it has multiple records in it
        for blast_record in blast_records:
            org_desig=file_name.split("_")[2].split("[")[0].replace(" ", "_")
            #properly formats the taxa id so we can use it to name things
            
            homo_sapiens = "[Homo sapiens]"
            blast_query=blast_record.query
            if homo_sapiens in blast_query:
                gene_name=blast_record.query.split("|")[4].split("[")[0].replace(" ", "_").replace("_protein", "").replace("_isoform_b3", "")
                formated_gene_name = gene_name[1:-1]
            else:
                formated_gene_name=blast_query.split("|")[4].split("_")[0]
            #this conditional properly formats the gene name so we can use it to name things
                #Basically there are 2 formats of sequence names I used to do the search
                #this statement switches the naming convention depending on which is used
                #it's not very general, so in the future I will make sure to properly name my search sequences
                #that should make this statement unnecessary 
            
            
            output_file_name=("gene_"+formated_gene_name +"_tiny_subset"+ "_summary.csv")
            #added subset here to see if we can limit the number of results to the top hits
            output_file=open(output_file_name, "a")
            
            #this puts together a nicely named file for each gene and species
            
            alignment_file_contents=""
            #setting up an empty list to hold the output of the following loop
            for alignment in blast_record.alignments[:3]:

                score_counter=[]
                e_val_counter=[]

                for hsp in alignment.hsps:
                    evalue=hsp.expect
                    e_val_counter.append(evalue)
                
                seq_designation = alignment.title.split("|")
                gi_number = seq_designation[1]
                ref_number = seq_designation[3]
                annotation = seq_designation[4]
                #breaking apart the annoation of the sequence name
                #otherwise we'd have way too many delimiters within delimiters
                output_line = str(gi_number + ","+ 
                                  ref_number + "," +
                                  annotation+ "," +
                                  str(min(e_val_counter)) + "\n")
                #this sets up the output for each alignment
                #had to alter the line breaks and parenthesis positioning
                
                alignment_file_contents = alignment_file_contents + output_line
                #this writes what we came up with to the holder string
            output_file.write(alignment_file_contents)
            #writing the output to the alignment file
            output_file.close()
        result_handle.close()

In [ ]:
reduced_subset_taxa_file = open("reduced_list_of_test_taxa.txt", "r")
reduced_subset_taxa = reduced_subset_taxa_file.read().splitlines()
parse_and_getID(reduced_subset_taxa)

# Example input

In [ ]:
example_contents = open("USH_Search_txid203908[ORGN].xml","r").read()
#example_contents

# Example Output

In [ ]:
example_output = open("gene_WHRN_tiny_subset_summary.csv").read()

# Grab Gene IDs from output with bash for each gene

```
cat gene_WHRN_tiny_subset_summary.csv | cut -d ',' -f 1 > WHRN_tiny_gi.txt
```

# Download full sequence for gene from NCBI

In [ ]:
def download_seqs_from_list_and_autoname(in_filename):    
    with open(in_filename, "r") as query_file:
        query_ids = query_file.read().splitlines()
    #open the file, grab the ids

    from Bio import Entrez
    Entrez.email = "hspeck@ucla.edu"
    # import Bio Entrez, let NCBI know who I am

    search_results = Entrez.read(Entrez.epost(db ="protein", 
                                              id = ",".join(query_ids)))  # needs the id's to be as a list separated by commas
    #upload the IDs to NCBI
    webenv_return = search_results["WebEnv"]
    query_key_return = search_results["QueryKey"]
    #grab the relevant variables to call on the stuff we posted

    count = len(query_ids)
    #assigns the count based on the number of sequences we searched for

    from urllib.error import HTTPError
    # load required library for the try and except conditions

    batch_size = 20
    #this determines how many things we retrieve and write to the file
    #can safely be larger in the future

    out_filename = str(in_filename[:-7]+".fasta")
    #attempting to rename the file based on the input of the original file
    out_handle = open(out_filename, "w")
    #open file to write to

    for start in range(0, count, batch_size):
        end = min(count, start+batch_size)
        print("Going to download record %i to %i" % (start+1, end))
        attempt = 0
        while attempt < 3:
            attempt += 1
            try:
                fetch_handle = Entrez.efetch(db="protein", #says which db
                                             rettype="fasta", #says what format the data should be in
                                             retmode="text", #what the output should be
                                             retstart = start, #say what range of results you want returned
                                             retmax = batch_size, #say end of range of results want returned
                                             webenv = webenv_return, #specify the info we uploaded with ePost
                                             query_key = query_key_return)
            except HTTPError as err:
                if 500 <= err.code <= 599:
                    print("Recieved error from server %s" % err)
                    print("Attempt %i of 3" % attempt)
                    time.sleep
                else:
                    raise
        data = fetch_handle.read()
        fetch_handle.close()
        out_handle.write(data)
    out_handle.close()

In [ ]:
download_seqs_from_list_and_autoname("WHRN_tiny_gi.txt")
#will give us the properly named input for the Multiple alignment

# Multiple Alignment by MUSCLE

In [ ]:
from Bio.Align.Applications import MuscleCommandline
muscle_cline = MuscleCommandline(input = "WHRN_tiny.fasta", out = "WHRN_tiny_Muscle_align.txt")
stdout, stder = muscle_cline()

# Run through RAxML

in shell
```
./raxmlHPC -m PROTGAMMAWAG -p 12345 -s Path/to/alignment/WHRN_tiny_Muscle_align.txt  -# 5 -n WHRNtn2
```

path depends on where RAxML is installed

Note that number of runs was reduced in order to have the tree completed for the presentation

We get a tree file out of this, feed it to R for the next part

But first:

# Format the Sequence Names For Tree making


Use Bash to grab the top lines of the Fasta lines

```
cat WHRN_tiny.fasta | grep ">" > WHRN_tiny_annotations.txt
```

In [ ]:
WHRN_annotations = open("WHRN_tiny_annotations.txt", "r").readlines()
name_file= open("WHRN_gene&sci&annotation.txt", "w")
for line in WHRN_annotations:
    #this handles formating of names
    #there are essentially a number of formats the genes can be name in, os it makes the naming process a bit tricky
    
    sequence_id = line.split(" ")[0].replace(">", "")
    #split the sequence ID out and get rid of FASTA formating
    
    
    formated_org = line.split("[")[1].split(' ')
    genus_code = formated_org[0][:3]
    genus = formated_org[0]
    species_code = formated_org[1][:3]
    final_org_name = genus_code + "_" + species_code
    #splits out the genus and species
    
    taxa_group = taxa_Dict[final_org_name]
    #assigns the species to a broad taxonomic division 
    
    gene_annotation_slice = line.split("[")[0].split(" ")[1:]
    if "PREDICTED:" in gene_annotation_slice:
        if "-like" in gene_annotation_slice:
            gene_annotation = gene_annotation_slice[0].replace(" ", "")
        else:
            gene_annotation = gene_annotation_slice[1:3]
    elif "Drosophila" in gene_annotation_slice:
        gene_annotation = gene_annotation_slice[1:3]
    else:
        gene_annotation = gene_annotation_slice[0:1]
    gene_annotation_final = "_".join(gene_annotation).replace(",", "")
    #cuts down the gene annoatation names to make them more manageable
    #problematic as the gene names do not conform to a single format that makes them easily parsable
    #have to switch between formats
    
    combined_name = '"' +final_org_name + " " + gene_annotation_final + '"'
    genus_and_gene = '"' +genus + " " + gene_annotation_final + '"'
    
    name_file.write(sequence_id+","+
                    final_org_name+","+
                    gene_annotation_final+","+ 
                    taxa_group+ ","+ 
                    combined_name+ "," +
                    genus_and_gene+'\n')
    #Writes it to the file
    
name_file.close()

# Run Through R

file at project/presentation_material/Presentation_R_file.R

```
rm(list = ls())
library(ape)
### Basic Tree!

WHRN_tree <- read.tree("/Path/o/best/Tree/RAxML_bestTree.WHRNtn2")
plot(WHRN_tree)
#bring in the tree from where ever it is and plot it

```
<img src = "files/WHRN_basic_tree.png">

### Make it nice and readable

```
# Read in and format the annotation data
WHRN_gene_annotations <- read.csv("WHRN_gene&sci&annotation.txt", 
                                  header =FALSE, 
                                  stringsAsFactors = FALSE)
names(WHRN_gene_annotations) <- c("GeneID", "OrgName", "Annotation", "Group", "CombinedName", "Genus_Gene")
rownames(WHRN_gene_annotations) <- WHRN_gene_annotations$GeneID



# Reorder data frame to match rows

order <- match(WHRN_tree$tip.label, rownames(WHRN_gene_annotations))
WHRN_gene_annotations <- WHRN_gene_annotations[,][order,]


WHRN_tree$tip.label <- WHRN_gene_annotations$Genus_Gene
# Rename tips of Tree to increase Readability

plot(WHRN_tree, cex = 0.5, label.offset = 0.1)
# decrease size of labels a bit

title("Whirlin (USH2D) best hits tree")
#add a title
add.scale.bar()
#add a scale bar!

# Color the tips to indicate which lineage the organism belongs to

WHRN_gene_annotations$Group <- as.factor(WHRN_gene_annotations$Group)

#start by setting the Group to factor
lineages <- unique(WHRN_gene_annotations$Group)
cols <- rainbow(n = length(lineages))
#color vector for legend
colvec <- cols[WHRN_gene_annotations$Group]
#color vector for tree
tiplabels(pch = 19, col = colvec)


# Add a legend!
legend(x = "bottomright", lwd =0, pch = 19 , legend = levels(lineages), col = cols, cex = 0.7)
```
Here is the output:



<img src ="files/Whirlin_presentation_color_tree.png">

Some clustering of predicted whirlins with Deuterostome whirlins, along with predicted but unannotated proteins

However, not all predicted whirlins cluster note the Octopus predicted whirlin

Annotated harmonin homologs clustering together